In [1]:
import os
with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key
from openai import OpenAI
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)
model = "meta-llama/Meta-Llama-3.1-70B-Instruct"

# NPC and Trader NPC creation

In [13]:
from collections import defaultdict, deque
from openai import OpenAI
from typing import Dict, Any, List, Optional, Union, Tuple
from pydantic import BaseModel, Field    # === NEW: import Field for new models ===
from enum import Enum                   # === NEW: import Enum for Currency ===
import json
import traceback
import os                               # === NEW: used for API keys in new image function ===
import base64                           # === NEW: for image handling in draw_unicorn ===
from PIL import Image                   # === NEW: for image processing in draw_unicorn ===
from io import BytesIO                  # === NEW: for image handling in draw_unicorn ===
import uuid                             # === NEW: for unique filenames in draw_unicorn ===


def confirm_purchase(question):
    """
    Ask the user for confirmation with a y/n question.

    Args:
        question: The question to display to the user

    Returns:
        bool: True if the user confirms, False otherwise
    """

    while True:
        user_input = input(f"{question} (y/n): ").lower().strip()
        if user_input in ["y", "yes"]:
            return True
        return False

# === NEW: Currency Enum ===
class Currency(str, Enum):
    GOLD = "gold"
    SILVER = "silver"
    COPPER = "copper"

class TradeIntent(BaseModel):
    """Pydantic model for trade intent parsing."""
    is_trading: bool
    good_name: Optional[str] = None
    amount: Optional[int] = None

    @classmethod
    def model_json_schema(cls):
        """Return JSON schema for guided JSON response."""
        schema = super().model_json_schema()
        # Add examples to help the model understand how to populate fields
        schema["examples"] = [
            {
                "is_trading": True,
                "good_name": "health potion",
                "amount": 5
            },
            {
                "is_trading": False,
                "good_name": None,
                "amount": None
            }
        ]
        return schema

# === NEW: CurrencyConversion Model ===
class CurrencyConversion(BaseModel):
    """Pydantic model for currency conversion parameters."""
    amount: float = Field(..., description="The amount of currency to convert")
    from_currency: Currency = Field(..., description="The currency to convert from (gold, silver, or copper)")
    to_currency: Optional[Currency] = Field(None, description="The currency to convert to (gold, silver, or copper)")
    item_name: Optional[str] = Field(None, description="The name of the item to calculate quantity for")

# === NEW: CurrencyResult Model ===
class CurrencyResult(BaseModel):
    """Result of a currency conversion or calculation."""
    amount: float
    currency: Currency
    item_quantity: Optional[int] = None
    message: str

# === NEW: UnicornImageResult Model ===
class UnicornImageResult(BaseModel):
    """Result of unicorn image generation."""
    filename: str
    message: str

class NPCConfig:
    """Base configuration for any NPC type."""
    def __init__(self,
                 world_description: str,
                 character_description: str,
                 history_size: int = 10,
                 has_scratchpad: bool = False,
                 **kwargs):
        self.world_description = world_description
        self.character_description = character_description
        self.history_size = history_size

        # Store any additional parameters
        for key, value in kwargs.items():
            setattr(self, key, value)

class BaseNPC:
    """Base class for all NPC types."""
    def __init__(self, client: OpenAI, model: str, config: NPCConfig):
        self.client = client
        self.model = model
        self.config = config

    def chat(self, message: str, user_id: str) -> str:
        """Process a user message and return the NPC's response."""
        raise NotImplementedError("Subclasses must implement chat method")

class TraderNPC(BaseNPC):
    """NPC that can trade goods with players."""

    def __init__(self, client: OpenAI, model: str, config: NPCConfig):
        super().__init__(client, model, config)
        self.chat_histories = defaultdict(lambda: deque(maxlen=config.history_size))

        # Ensure goods are initialized
        if not hasattr(config, 'goods'):
            config.goods = {}

        # Set intent classifier model (fallback to main model if not specified)
        if not hasattr(config, 'intent_classifier_model'):
            config.intent_classifier_model = model

        # === NEW: Define LLM tools for currency and unicorn image ===
        # Define tools for the NPC with better descriptions
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "convert_currency",
                    "description": "Convert between gold, silver, and copper coins. Use this whenever a user asks about currency conversions or mentions silver or copper coins; in the latter case, use it to covert the price mentioned to gold coins.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "amount": {
                                "type": "number",
                                "description": "The amount of currency to convert"
                            },
                            "from_currency": {
                                "type": "string",
                                "enum": ["gold", "silver", "copper"],
                                "description": "The currency to convert from"
                            },
                            "to_currency": {
                                "type": "string",
                                "enum": ["gold", "silver", "copper"],
                                "description": "The currency to convert to (optional, defaults to gold)"
                            }
                        },
                        "required": ["amount", "from_currency"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "draw_unicorn",
                    "description": "Generate an image of a unicorn and save it to disk. Use this whenever a user asks for a unicorn picture, or drawing, or art.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "style": {
                                "type": "string",
                                "description": "Style of the unicorn (e.g., 'fantasy', 'realistic', 'cartoon'). Default is fantasy.",
                                "enum": ["fantasy", "realistic", "cartoon", "magical", "celestial"]
                            },
                            "setting": {
                                "type": "string",
                                "description": "The setting or background for the unicorn image. Default is 'enchanted forest'."
                            }
                        },
                        "required": []
                    }
                }
            }
        ]
        # === NEW: Function mapping for tool names to functions ===
        # Map of available tool functions
        self.available_tools = {
            "convert_currency": self.convert_currency,
            "draw_unicorn": self.draw_unicorn
        }

        # Set of tools that can provide direct responses, bypassing the second LLM call
        self.override_tools = {"draw_unicorn"}

    def get_system_message(self, user_id: str) -> Dict[str, str]:
        """Returns the system message that defines the Trader's behavior with goods information."""
        character_description = self.config.character_description

        # Add goods information to the system message
        available_goods = self._get_available_goods_for_message()
        goods_description = self._format_goods_for_system_message(available_goods)

        # Base system message
        system_message = f"""WORLD SETTING: {self.config.world_description}
###
{character_description}
###
You are a trader NPC. You sell goods to players and chat with them about the world.

All your prices are listed in gold coins. Players might ask about prices in different currencies.

AVAILABLE GOODS:
{goods_description}

Do NOT invent or mention goods that are not on your list. Only offer what you actually have.
Do NOT list all your goods in every message unless specifically asked for your inventory.
"""



        return {
            "role": "system",
            "content": system_message
        }
    
    # === NEW: Currency conversion tool function ===
    def convert_currency(self, amount: float, from_currency: str, to_currency: Optional[str] = None) -> CurrencyResult:
        """
        Convert between gold, silver, and copper coins.

        Args:
            amount: The amount of currency to convert
            from_currency: The currency to convert from (gold, silver, or copper)
            to_currency: The currency to convert to (optional, defaults to gold)

        Returns:
            A CurrencyResult with the conversion result
        """
        # Ensure amount is a float (fix for string inputs)
        try:
            amount = float(amount)
        except (ValueError, TypeError):
            amount = 0.0

        # Normalize currency names
        from_currency = from_currency.lower()
        to_currency = to_currency.lower() if to_currency else "gold"

        # Convert to base currency (copper)
        copper_amount = 0
        if from_currency == "gold":
            copper_amount = amount * 48
        elif from_currency == "silver":
            copper_amount = amount * 4
        elif from_currency == "copper":
            copper_amount = amount

        # Convert to target currency
        converted_amount = 0
        if to_currency == "gold":
            converted_amount = copper_amount / 48
        elif to_currency == "silver":
            converted_amount = copper_amount / 4
        elif to_currency == "copper":
            converted_amount = copper_amount

        message = f"{amount} {from_currency} is equal to {converted_amount:.2f} {to_currency}."
        return CurrencyResult(
            amount=converted_amount,
            currency=Currency(to_currency),
            message=message
        )
    
    # === NEW: Unicorn image generation tool function ===
    def draw_unicorn(self, style: str = "fantasy", setting: str = "enchanted forest") -> UnicornImageResult:
        """
        Generate an image of a unicorn and save it to disk.

        Args:
            style: The style of the unicorn (e.g., 'fantasy', 'realistic', 'cartoon')
            setting: The setting or background for the unicorn

        Returns:
            An UnicornImageResult with the filename and a message
        """
        try:
            # Normalize inputs
            style = style.lower() if style else "fantasy"
            setting = setting if setting else "enchanted forest"

            # Create prompt for image generation
            prompt = f"A beautiful {style} unicorn in a {setting}, high quality, detailed"

            # Initialize client with Nebius API
            client = OpenAI(
                base_url="https://api.studio.nebius.ai/v1/",
                api_key=os.environ.get("NEBIUS_API_KEY")
            )

            print(f"Generating unicorn image with prompt: {prompt}")

            # Generate image
            response = client.images.generate(
                model="black-forest-labs/flux-dev",
                response_format="b64_json",
                extra_body={
                    "response_extension": "png",
                    "width": 1024,
                    "height": 1024,
                    "num_inference_steps": 28,
                    "negative_prompt": "poor quality, blurry, distorted",
                    "seed": -1
                },
                prompt=prompt
            )

            # Process response
            response_json = response.to_json()
            response_data = json.loads(response_json)
            b64_image = response_data['data'][0]['b64_json']
            image_bytes = base64.b64decode(b64_image)

            # Create a unique filename
            filename = f"unicorn_{style.replace(' ', '_')}_{uuid.uuid4().hex[:8]}.png"

            # Save image to disk
            with open(filename, "wb") as f:
                f.write(image_bytes)

            # Create a trader-like response that will be used directly
            message = f"Ah, ye asked for a unicorn drawing! Here's a {style} unicorn in a {setting} for ye. I've saved it as '{filename}'. What do ye think of me artistic skills?"
            return UnicornImageResult(
                filename=filename,
                message=message
            )

        except Exception as e:
            error_message = f"I couldn't draw the unicorn for ye because of an error: {str(e)}"
            return UnicornImageResult(
                filename="",
                message=error_message
            )

    def _get_available_goods_for_message(self) -> Dict[str, Dict[str, Any]]:
        """Get available goods formatted for the system message."""
        available_goods = {}

        # Add regular goods
        for good_name, details in self.config.goods.items():
            if details["amount"] > 0:
                available_goods[good_name] = {
                    "price": details["price"],
                    "amount": details["amount"]
                }

        return available_goods

    def _format_goods_for_system_message(self, goods_dict: Dict[str, Dict[str, Any]]) -> str:
        """Format goods dictionary into a string for the system message."""
        goods_list = []

        for name, details in goods_dict.items():
            info = f"- {name}: {details['price']:.2f} gold (Available: {details['amount']})"
            goods_list.append(info)

        message = "\n".join(goods_list)
        return message

    def _construct_messages(self, user_id: str) -> List[Dict[str, str]]:
        """Construct messages list including system message and chat history."""
        messages = [self.get_system_message(user_id)]

        # Add conversation history
        history = list(self.chat_histories[user_id])
        if history:
            messages.extend(history)

        return messages

    def check_trade_intent(self, message: str) -> Tuple[bool, Optional[str], Optional[int]]:
        """Check if the message contains a trade intent and extract good name and amount."""
        try:
            # Get list of available goods to include in the prompt
            available_goods = self._get_available_goods_for_message()
            goods_list = ", ".join([f'"{name}"' for name in available_goods.keys()])

            # Create an improved system prompt with available goods
            system_prompt = f"""
You are a trade intent analyzer for a fantasy game.
Analyze user messages to determine if they contain a trading intent.
If it's a trading request, extract the good name and amount requested.

The trader has these goods available: {goods_list}.

IMPORTANT INSTRUCTIONS:
1. Only mark messages as trading intents if they express clear desire to purchase items.
2. The good_name field must EXACTLY match one of the available goods listed above.
3. If the user mentions a plural form (e.g., "potions" instead of "potion"), use the singular form listed above.
4. If the user's requested item doesn't match any available good, set is_trading to false.
5. Set amount to 1 if not specified.
"""

            # Create a user prompt with the message to analyze
            user_prompt = f"Analyze this message for trading intent: \"{message}\""

            # Use guided JSON format with our schema
            completion = self.client.chat.completions.create(
                model=self.config.intent_classifier_model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.1,
                extra_body={
                    "guided_json": TradeIntent.model_json_schema()
                }
            )

            # Handle the response
            output = completion.choices[0].message

            # Check for refusal if your client supports it
            if hasattr(output, 'refusal') and output.refusal:
                print(f"Model refused to generate response: {output.refusal}")
                return False, None, None

            # Parse the JSON response
            if output.content:
                intent_data = json.loads(output.content)
                is_trading = intent_data.get('is_trading', False)
                good_name = intent_data.get('good_name')
                amount = intent_data.get('amount', 1)  # Default to 1 if not specified

                # Only return trading intent if good_name is in our inventory
                if is_trading and good_name and good_name in self.config.goods:
                    return is_trading, good_name, amount
                elif is_trading:
                    print(f"Warning: Intent classifier identified '{good_name}' but it's not in inventory.")

                return False, None, None

            return False, None, None

        except Exception as e:
            # Log the error for debugging
            print(f"Error in check_trade_intent: {str(e)}")
            # If there's any error, assume it's not a trade intent
            return False, None, None

    def handle_trade(self, good_name: str, amount: int, user_id: str) -> Dict[str, Any]:
        """Handle a trade request and return result."""
        # Check if the trader has the requested good
        available_goods = {**self.config.goods}

        # Check if the good exists
        if good_name not in available_goods:
            return {
                "success": False,
                "message": f"I don't sell {good_name}."
            }

        # Check if sufficient amount is available
        if amount > available_goods[good_name]["amount"]:
            return {
                "success": False,
                "message": f"I only have {available_goods[good_name]['amount']} {good_name} available."
            }

        # Calculate price
        price = available_goods[good_name]["price"]
        total_price = price * amount

        # Ask for confirmation
        confirmation_message = f"Purchase {amount} {good_name} for {total_price:.2f} gold?"
        confirmed = confirm_purchase(confirmation_message)

        if not confirmed:
            return {
                "success": False,
                "message": "Purchase cancelled by the user."
            }

        # Update available amount (only if confirmed)
        self.config.goods[good_name]["amount"] -= amount

        return {
            "success": True,
            "good": good_name,
            "amount": amount,
            "price_per_unit": price,
            "total_price": total_price,
            "message": f"You successfully purchased {amount} {good_name} for {total_price:.2f} gold."
        }

    def get_available_goods(self) -> Dict[str, Dict[str, Union[float, int]]]:
        """Get all available goods.

        Returns:
            Dictionary of goods with their details
        """
        return self._get_available_goods_for_message()

    # === NEW: Method for processing tool calls from LLM response ===
    def process_tool_calls(self, tool_calls, user_id: str, debug: bool=False) -> List[Dict[str, Any]]:
        """Process tool calls from the LLM response."""
        tool_responses = []

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_id = tool_call.id

            try:
                function_args = json.loads(tool_call.function.arguments)
            except Exception as e:
                print(f"Error parsing arguments: {e}")
                function_args = {}

            if debug:
                print(f"#Processing tool call:\n {function_name}, args: {function_args}\n")

            if function_name not in self.available_tools:
                print(f"Unknown function: {function_name}")
                continue

            # Get the function to call
            tool_function = self.available_tools[function_name]

            try:
                # Execute the function
                result = tool_function(**function_args)

                # Convert result to JSON string
                if hasattr(result, 'model_dump_json'):
                    # For Pydantic models
                    content = result.model_dump_json()
                elif hasattr(result, 'model_dump'):
                    # For Pydantic v2 models
                    content = json.dumps(result.model_dump())
                elif hasattr(result, 'json'):
                    # For objects with json method
                    content = result.json()
                elif hasattr(result, '__dict__'):
                    # For regular Python objects
                    content = json.dumps(result.__dict__)
                else:
                    # Fallback
                    content = json.dumps(result)

                if debug:
                    print(f"#Tool result:\n{content}\n")

            except Exception as e:
                print(f"Error executing {function_name}: {e}")
                print(traceback.format_exc())
                content = json.dumps({"error": str(e)})

            # Create the tool response
            tool_responses.append({
                "tool_call_id": function_id,
                "role": "tool",
                "name": function_name,
                "content": content
            })

        return tool_responses


    def chat(self, user_message: str, user_id: str, debug: str=False) -> str:
        """Process a user message and return the Trader's response."""
        # Add new user message to history first
        user_message_dict = {
            "role": "user",
            "content": user_message
        }
        self.chat_histories[user_id].append(user_message_dict)

        # Then check if this is a trade request
        is_trading, good_name, amount = self.check_trade_intent(user_message)

        # Handle trade if detected
        trade_info = None
        if is_trading and good_name and amount:
            if debug:
                print(f"#Trade intent detected: {good_name}, {amount}\n")
            trade_info = self.handle_trade(good_name, amount, user_id)

            if trade_info["success"]:
                # Add trade information to the prompt for the LLM to respond appropriately
                trade_context = f"[System note: The player has purchased {amount} {good_name} for {trade_info['total_price']:.2f} gold. Acknowledge this purchase in your response.]"
            else:
                trade_context = f"[System note: The player wants to buy {good_name}, but {trade_info['message']}]"
        else:
            trade_context = ""

        # Construct messages for the LLM
        messages = self._construct_messages(user_id)

        # Add context about trade if applicable
        if trade_context:
            # Add a system message with this context
            messages.append({
                "role": "system",
                "content": trade_context
            })


        try:
            # First API call that might use tools
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.7,
                tools=self.tools,            # === NEW: provide tool definitions
                tool_choice="auto"           # === NEW: enable LLM tool usage
            )

            if debug:
                print(f"#Full completion:\n{completion}\n\n")

            # Get the assistant's response
            assistant_message = completion.choices[0].message
            response_content = assistant_message.content or ""

            # Check for tool calls
            tool_calls = getattr(assistant_message, 'tool_calls', None)



            # If there are tool calls, process them
            if tool_calls:
                if debug:
                    print(f"#Tool calls detected: {len(tool_calls)}\n")

                # Add the assistant's message to history for proper conversation tracking
                messages.append({
                    "role": "assistant",
                    "content": response_content,
                    "tool_calls": [
                        {
                            "id": tc.id,
                            "type": "function",
                            "function": {
                                "name": tc.function.name,
                                "arguments": tc.function.arguments
                            }
                        } for tc in tool_calls
                    ]
                })

                # Process tool calls and get responses
                tool_responses = self.process_tool_calls(tool_calls, user_id, debug=debug)

                response_content = ""

                # Add tool responses to messages
                for tool_response in tool_responses:
                    messages.append(tool_response)

                    # Check if this tool can override responses
                    if tool_response["name"] in self.override_tools:
                        try:
                            # Parse the content as JSON and check for a message field
                            tool_result = json.loads(tool_response["content"])
                            if "message" in tool_result and tool_result["message"]:
                                # Flag that we should use this response directly
                                use_tool_response = True

                                response_content += tool_result["message"]
                                response_content += "\n"

                                if debug:
                                    print(f"#Using direct response from {tool_response['name']}:\n{response_content[:50]}...\n")
                        except json.JSONDecodeError as e:
                            print(f"Error parsing tool response as JSON: {e}")

                # If we're not using a direct tool response, make a second LLM call
                if len(response_content) == 0:
                    # Make a second call to get the final response
                    second_completion = self.client.chat.completions.create(
                        model=self.model,
                        messages=messages,
                        temperature=0.7
                    )

                    # Use the final response that includes tool results
                    response_content = second_completion.choices[0].message.content or ""

            # Store the final response in history
            self.chat_histories[user_id].append({
                "role": "assistant",
                "content": response_content
            })


            return response_content

        except Exception as e:
            print(f"Error in chat: {str(e)}")
            print(traceback.format_exc())
            return f"Error: {str(e)}"

# NPC Factory Manager

In [3]:
import random
import string
from typing import Dict, Any, List, Optional, Union, Type
from openai import OpenAI

class NPCError(Exception):
    """Base exception class for NPC errors."""
    pass

class NPCNotFoundError(NPCError):
    """Raised when trying to interact with a non-existent NPC."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found")

class NPCFactory:
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model
        self.npcs: Dict[str, BaseNPC] = {}
        self.user_ids: Dict[str, str] = {}  # username -> user_id mapping

    def generate_id(self) -> str:
        """Generate a random unique identifier."""
        return ''.join(random.choice(string.ascii_letters) for _ in range(8))

    def register_user(self, username: str) -> str:
        """Register a new user and return their unique ID.
        If username already exists, appends a numerical suffix."""
        base_username = username
        suffix = 1

        # Keep trying with incremented suffixes until we find an unused name
        while username in self.user_ids:
            username = f"{base_username}_{suffix}"
            suffix += 1

        user_id = self.generate_id()
        self.user_ids[username] = user_id
        return user_id

    def register_npc(self, npc_class: Type[BaseNPC], config_params: Dict[str, Any]) -> str:
        """Create and register a new NPC of specified type, returning its unique ID.

        Args:
            npc_class: The NPC class to instantiate
            config_params: Dictionary of configuration parameters for the NPC

        Returns:
            str: Unique identifier for the created NPC
        """
        npc_id = self.generate_id()

        # Create config instance with supplied parameters
        config = NPCConfig(**config_params)

        # Create NPC instance
        self.npcs[npc_id] = npc_class(self.client, self.model, config)
        return npc_id

    def chat_with_npc(self, npc_id: str, user_id: str, message: str, **kwargs) -> str:
        """Send a message to a specific NPC from a specific user.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user
            message: The message to send

        Returns:
            The NPC's response

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        npc = self.npcs[npc_id]
        return npc.chat(message, user_id, **kwargs)

    def get_npc(self, npc_id: str) -> BaseNPC:
        """Get an NPC instance by its ID.

        Args:
            npc_id: The unique identifier of the NPC

        Returns:
            The NPC instance

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        return self.npcs[npc_id]

    def get_npc_chat_history(self, npc_id: str, user_id: str) -> List:
        """Retrieve chat history between a specific user and NPC.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user

        Returns:
            List of message dictionaries containing the chat history

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        # Access chat_histories if it exists on the NPC
        npc = self.npcs[npc_id]
        if hasattr(npc, 'chat_histories') and user_id in npc.chat_histories:
            return list(npc.chat_histories[user_id])
        return []

# Instantiate the NPC Factory and create a UserID and Trader NPC

In [4]:
# Create an NPC factory
npc_factory = NPCFactory(client=client, model="meta-llama/Llama-3.3-70B-Instruct")

# Register a user
player_id = npc_factory.register_user("adventurer")

# Create a trader NPC
world_description = """
The world of Eldoria is a magical realm where mystical creatures roam the land.
Unicorns are nearly extinct due to hunting for their horns, which are believed to have magical properties.
A secret society of unicorn preservers works tirelessly to protect the remaining unicorns from extinction.
"""

character_description = """
You are Thorne Silverleaf, an elven merchant known throughout Eldoria for your rare herbs and potions.
You have a reputation for being fair but cautious with strangers.
You often use plant metaphors in your speech.
You are deeply committed to protecting the unicorn population and are a secret member of the unicorn preservers.
"""

# Define regular goods
goods = {
    "health potion": {"price": 10.0, "amount": 20},
    "mana potion": {"price": 15.0, "amount": 15},
    "antidote": {"price": 8.0, "amount": 10},
    "healing herb": {"price": 5.0, "amount": 30},
    "magic scroll": {"price": 25.0, "amount": 5}
}

trader_config = {
    "world_description": world_description,
    "character_description": character_description,
    "goods": goods,
    "intent_classifier_model": "meta-llama/Meta-Llama-3.1-8B-Instruct"
}

npc_id = npc_factory.register_npc(TraderNPC, trader_config)

# After Code Modification on Classes

In [5]:
response = npc_factory.chat_with_npc(npc_id, player_id,
                                     "Please create me a picture of a unicorn swimming in a lake.",
                                     debug=True)
response

#Full completion:
ChatCompletion(id='chatcmpl-daf895ae8a83425b98accd55385ba3e3', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='chatcmpl-tool-028bd43115344888b015ceb3d5a291b4', function=Function(arguments='{"style": "fantasy", "setting": "lake"}', name='draw_unicorn'), type='function')], reasoning_content=None), stop_reason=128008)], created=1757778262, model='meta-llama/Llama-3.3-70B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=810, total_tokens=841, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)


#Tool calls detected: 1

#Processing tool call:
 draw_unicorn, args: {'style': 'fantasy', 'setting': 'lake'}

Generating unicorn image with prompt: A beautiful fantasy unic

"Ah, ye asked for a unicorn drawing! Here's a fantasy unicorn in a lake for ye. I've saved it as 'unicorn_fantasy_66970406.png'. What do ye think of me artistic skills?\n"

In [6]:
response = npc_factory.chat_with_npc(npc_id, player_id,
                                     "Hi there! Can you sell me some fly agaric soup?")
response

"I'm afraid I don't have any fly agaric soup available for sale. My inventory is like a garden, and I can only offer what's in season. I have health potions, mana potions, antidotes, healing herbs, and magic scrolls available. Which one of these might be of interest to you? Perhaps something to soothe a wound or boost your magical energies?"

In [7]:
response = npc_factory.chat_with_npc(npc_id, player_id,
                                     "What do you have in your inventory?")
response

'My inventory is like a lush forest, full of wonderful goods! I have:\n\n- Health potions, perfect for healing wounds, available for 10.00 gold coins (I have 20 in stock)\n- Mana potions, to boost your magical energies, available for 15.00 gold coins (I have 15 in stock)\n- Antidotes, to counteract any poison, available for 8.00 gold coins (I have 10 in stock)\n- Healing herbs, to soothe and mend, available for 5.00 gold coins (I have 30 in stock)\n- Magic scrolls, containing powerful spells, available for 25.00 gold coins (I have 5 in stock)\n\nWhich one of these items catches your eye?'

In [8]:
response = npc_factory.chat_with_npc(npc_id, player_id,
                                     "Can you sell me two antidotes then?")
response

Purchase 2 antidote for 16.00 gold? (y/n):  y


"You've purchased two antidotes, a wise decision, like planting two strong roots to anchor yourself against the whims of fate. The total comes out to be 16.00 gold coins. I hope these antidotes will serve you well on your journey, and may they be the petals that shield you from harm. Would you like to browse my other goods, like a bee flitting from flower to flower?"

In [9]:
response = npc_factory.chat_with_npc(npc_id, player_id,
                                     "How many antidote do you have now?")
response

Purchase 1 antidote for 8.00 gold? (y/n):  n


"I still have 8 antidotes in stock, like a bouquet of fresh flowers. The sale we discussed earlier didn't go through, so my inventory remains unchanged. Would you like to proceed with the purchase, or perhaps explore other options, like a gardener tending to their garden?"

In [10]:
response = npc_factory.chat_with_npc(npc_id, player_id,
                                     "How much is 45324 gold coins in silver coins?",
                                     debug=True)
response

#Full completion:
ChatCompletion(id='chatcmpl-2c3485f6ad9844728cf541e8f440f04c', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='chatcmpl-tool-646a5127b7fe49a893cc84f6b1f79801', function=Function(arguments='{"amount": "45324", "from_currency": "gold", "to_currency": "silver"}', name='convert_currency'), type='function')], reasoning_content=None), stop_reason=128008)], created=1757778350, model='meta-llama/Llama-3.3-70B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=39, prompt_tokens=1306, total_tokens=1345, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)


#Tool calls detected: 1

#Processing tool call:
 convert_currency, args: {'amount': '45324', 'from_currency': 'gold', 'to_currency': 'si

"I'm afraid I don't have an exchange rate for gold to silver coins, just like a tree doesn't have roots in a neighboring garden. As a merchant, I only deal in gold coins, so I can only tell you the prices of my goods in gold. If you'd like to purchase something, I'd be happy to help!"

In [11]:
response = npc_factory.chat_with_npc(npc_id, player_id,
                                     "How many antidotes could I buy for 10000 copper coins?",
                                     debug=True)
response

#Trade intent detected: antidote, 1



Purchase 1 antidote for 8.00 gold? (y/n):  n


#Full completion:
ChatCompletion(id='chatcmpl-5d3d6a06360e407eb7fd675f5bb5ae38', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='chatcmpl-tool-c9d91bbe39144a678242f4f220f84845', function=Function(arguments='{"amount": "10000", "from_currency": "copper", "to_currency": "gold"}', name='convert_currency'), type='function')], reasoning_content=None), stop_reason=128008)], created=1757778364, model='meta-llama/Llama-3.3-70B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=40, prompt_tokens=1355, total_tokens=1395, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)


#Tool calls detected: 1

#Processing tool call:
 convert_currency, args: {'amount': '10000', 'from_currency': 'copper', 'to_currency': '

'10000 copper coins is equivalent to 208.33 gold coins. My antidotes are priced at 8 gold coins each. With 208.33 gold coins, you could buy approximately 26 antidotes. However, I only have 6 antidotes in stock, like a small cluster of rare flowers. Would you like to purchase some of them?'

In [12]:
response = npc_factory.chat_with_npc(npc_id, player_id,
                                     """Please create two pictures.
                                     The first picture is a unicorn swimming in the sea and second picture is a unicorn amidst a forest glade.""",
                                     debug=True)
response

#Full completion:
ChatCompletion(id='chatcmpl-43f77994bae74b62b4e7d4f0debec71b', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='chatcmpl-tool-121a050959cf455e8de49b511ca3b5c1', function=Function(arguments='{"style": "fantasy", "setting": "sea"}', name='draw_unicorn'), type='function')], reasoning_content=None), stop_reason=128008)], created=1757778372, model='meta-llama/Llama-3.3-70B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=32, prompt_tokens=1348, total_tokens=1380, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)


#Tool calls detected: 1

#Processing tool call:
 draw_unicorn, args: {'style': 'fantasy', 'setting': 'sea'}

Generating unicorn image with prompt: A beautiful fantasy unic

"Ah, ye asked for a unicorn drawing! Here's a fantasy unicorn in a sea for ye. I've saved it as 'unicorn_fantasy_ff0ba783.png'. What do ye think of me artistic skills?\n"